# **Instalción de librerias**

In [1]:
!pip install torchsummary
!pip install torchmetrics
!pip install torchvision
!pip install opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.3/802.3 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.7/463.7 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 13.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.2/779.2 MB 16.1 MB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 14.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 18.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1
  Attempting uninstall: triton
    Found existing installation: triton 2.1.0
    Uninstalling triton-2.1.0:
      Successfully uninstalled triton-2.1.0
  Attempting uninstall: nvidia-nccl-cu12
    Fou

In [ ]:
!python eval_noise.py

# **Testing Noise**

Uno de los objetivos de este proyecto es la posterior implementación de estos modelos en una FPGA,  con el fin de que sea capaz de resolver problemas de detección desde un satélite.

Se sabe que los modelos en una FPGA están expuestos a ruido, este ruido puede afectar en cualquier parte del modelo, se han considerado los siguientes casos:
* **Caso 1**: Analizamos el ruido que afecta a la entrada del modelo, como consecuencia del ruido que se recibe en la captura de de la señal de entrada, debido a las inferencias electrónicas.
* **Caso 2**: se pretende simular el ruido que puede ocurrir dentro del modelo, es decir, entre las conexiones de capas.
* **Caso 3**: se simula el ruido que puede afectar a los pesos del modelo debido a las fluctuaciones que pueden surgir en el almacenamiento de estos.

Para el caso 2 y 3 de análisis de ruido se ha considerado el caso, de que en caso de que exista ruido este se produce con la inyección de un 1 o un 0, en algún punto aleatorio de los pesos o activaciones.

En este ***Notebook*** se pretende analizar diferentes casos, de como afecta los diferentes ruidos de manera aislada y como se ve afectado el modelo en caso de que se produzca el ruido de manera simultanea en los dos o tres casos. Los resutlados obtenidos son guardados en la carpeta correspondiente del modelo que está siendo ejecutado, en la subcarpeta de ***results_noise***, en el que son analizadas las métricas de loss y mAP50.

Como en los ***Notebooks*** anteriores, la ejecución de estos códigos hay que tener en cuenta que dependen directamente del fichero de configuración ***config.py***, para la selección del modelo. En este cuaderno a diferencia de los otros no ejecutamos el código con un !python eval_noise.py, si no que hacemos un import, por comodidad de este código y poder evaluar los diferentes casos. Siendo así el código, si quieres probar diferentes modelos con el fichero ***config.py*** hay que reiniciar el kernel despues de modificar el fichero.

Si se quiere realizar las pruebas de **noise.py** sobre los modelos optimizados se tiene que especificar la ruta del modelo optimizado aunque tiene que tener coherencia con **config.py**

In [1]:
from eval_noise import test_noise

/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Para cada uno de los casos hay que introducir diferentes niveles de ruido, desde niveles bajos hasta ir subiendolos progresivamente. Hay que tener en cuenta que en cada caso, el modelo se ve influenciado de una manera u otra, entonces estos niveles de ruido tienen diferente nivel en cada uno de ellos.

## **Test case1**

In [2]:
case1=True
case2=False
percentage_layers_case_1  = [0, 0.05, 0.1, 0.12, 0.15, 0.2, 0.25, 0.3] if case1 else [0]
slices_case1 = [3,3]

test_noise( case1=case1, percentage_layers_case_1=percentage_layers_case_1, slices_case1 = slices_case1,
            case2=case2, percentage_layers_case_2=[0], slices_case2 = [0,0])

Test with case1: True, case2: False
=> Loading checkpoint
=> Loading checkpoint
Model loaded: tinyissimoYOLO/Adam_None_Airbus_256_BATCH_32_LR_0.0005/model/37_YOLO_best.pth.tar
Noise level 1: Slice: [0, 3], Percentage: 0


RuntimeError: OrderedDict mutated during iteration

## **Test case 3**

In [4]:
case1=False
case2=True
percentage_layers_case_2  = [0, 0.05, 0.1, 0.12, 0.15, 0.2, 0.25, 0.3] if case2 else [0]
slices_case2 = [3,3]

test_noise(case1=case1, percentage_layers_case_1=[0], slices_case1 = [0,0],
           case2=case2, percentage_layers_case_1=percentage_layers_case_2, slices_case2 = slices_case2)

Test with case1: False, case2: False, case3: True
File case3_results_noise.txt exists. Deleting...
=> Loading checkpoint
Model loaded: efficientnet/Adam_None_Airbus_256_BATCH_32_LR_0.0005/model/93_YOLO_best.pth.tar
Noise level 3: Slice: [0, 4], Percentage: 0
Number of layers affected: 0


Eval: Valid: : 100%|██████████| 88/88 [00:07<00:00, 12.33it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
13.290      |5.927       |4.004       |3.293       |0.065       |
---------------------------------------------------
Valid: 	 mAP@50: 0.836296, mAP@75: 0.495590, mAP@90: 0.015652, Mean Loss: 13.289885
Noise level 3: Slice: [1, 4], Percentage: 0
Number of layers affected: 0


Eval: Valid: : 100%|██████████| 88/88 [00:06<00:00, 13.15it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
13.290      |5.927       |4.004       |3.293       |0.065       |
---------------------------------------------------
Valid: 	 mAP@50: 0.836296, mAP@75: 0.495590, mAP@90: 0.015652, Mean Loss: 13.289885
Noise level 3: Slice: [2, 4], Percentage: 0
Number of layers affected: 0


...

---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
14.847      |6.481       |4.444       |3.852       |0.070       |
---------------------------------------------------
Valid: 	 mAP@50: 0.809272, mAP@75: 0.434536, mAP@90: 0.012453, Mean Loss: 14.846798
Noise level 3: Slice: [2, 4], Percentage: 0.2
Number of layers affected: 13


Eval: Valid: : 100%|██████████| 88/88 [00:06<00:00, 13.14it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
14.845      |6.494       |4.444       |3.836       |0.070       |
---------------------------------------------------
Valid: 	 mAP@50: 0.811177, mAP@75: 0.445842, mAP@90: 0.010801, Mean Loss: 14.845245
Noise level 3: Slice: [3, 4], Percentage: 0.2
Number of layers affected: 9


Eval: Valid: : 100%|██████████| 88/88 [00:06<00:00, 13.02it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
14.845      |6.494       |4.444       |3.836       |0.070       |
---------------------------------------------------
Valid: 	 mAP@50: 0.811173, mAP@75: 0.445827, mAP@90: 0.010800, Mean Loss: 14.845205


Estoy viendo que en cuanto modificamos los pesos, en uno de los bits mas significativos, el modelo es muy sensible. Interesa tener activado el seed? 

## **Test case1 y case 2**

In [6]:
case1=True
case2=True
case3=False
noise_level_case_1  = [0, 0.01, 0.03, 0.05, 0.07, 0.08, 0.1, 0.125 ,0.15, 0.2, 0.3] if case1 else [0]
noise_level_case_2  = [0, 0.1, 0.2, 0.3, 0.5] if case2 else [0]
test_noise(case1 = case1, noise_level_case_1 = noise_level_case_1, 
           case2 = case2, noise_level_case_2 = noise_level_case_2,
           case3 = case3, noise_level_case_3 = [0])

Test with case1: True, case2: True, case3: False
File case1_case2_results_noise.txt exists. Deleting...
=> Loading checkpoint
Model loaded: efficientnet/Adam_None_Airbus_256_BATCH_32_LR_0.0005/model/93_YOLO_best.pth.tar
Noise level 1: 0
Noise level 2: 0


Eval: Valid: : 100%|██████████| 88/88 [00:06<00:00, 13.89it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
13.087      |5.913       |4.003       |3.105       |0.065       |
---------------------------------------------------
Valid: 	 mAP@50: 0.836296, mAP@75: 0.495590, mAP@90: 0.015712, Mean Loss: 13.086824
Noise level 1: 0
Noise level 2: 0.1


Eval: Valid: : 100%|██████████| 88/88 [00:06<00:00, 14.40it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
31.516      |8.422       |4.301       |18.475      |0.318       |
---------------------------------------------------
Valid: 	 mAP@50: 0.824538, mAP@75: 0.348066, mAP@90: 0.004723, Mean Loss: 31.516115
Noise level 1: 0
Noise level 2: 0.2


...

---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
140.123     |52.635      |23.606      |62.531      |1.352       |
---------------------------------------------------
Valid: 	 mAP@50: 0.000067, mAP@75: 0.000000, mAP@90: 0.000000, Mean Loss: 140.122789
Noise level 1: 0.3
Noise level 2: 0.3


Eval: Valid: : 100%|██████████| 88/88 [00:12<00:00,  7.16it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
231.140     |64.861      |25.020      |138.723     |2.536       |
---------------------------------------------------
Valid: 	 mAP@50: 0.000168, mAP@75: 0.000001, mAP@90: 0.000000, Mean Loss: 231.140104
Noise level 1: 0.3
Noise level 2: 0.5


Eval: Valid: : 100%|██████████| 88/88 [00:55<00:00,  1.57it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
517.399     |98.216      |29.659      |383.350     |6.174       |
---------------------------------------------------
Valid: 	 mAP@50: 0.000330, mAP@75: 0.000001, mAP@90: 0.000000, Mean Loss: 517.398553


## **Test case 1 y case 3**

In [8]:
case1=True
case2=False
case3=True
noise_level_case_1  = [0, 0.01, 0.03, 0.05, 0.07, 0.08, 0.1, 0.125 ,0.15, 0.2, 0.3] if case1 else [0]
noise_level_case_3 = [0, 0.001, 0.0025, 0.005] if case3 else [0]
test_noise(case1 = case1, noise_level_case_1 = noise_level_case_1, 
           case2 = case2, noise_level_case_2 = [0],
           case3 = case3, noise_level_case_3 = noise_level_case_3)

Test with case1: True, case2: False, case3: True
File case1_case3_results_noise.txt exists. Deleting...
=> Loading checkpoint
Model loaded: efficientnet/Adam_None_Airbus_256_BATCH_32_LR_0.0005/model/93_YOLO_best.pth.tar
Noise level 1: 0
Noise level 3: 0


Eval: Valid: : 100%|██████████| 88/88 [00:06<00:00, 13.71it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
13.290      |5.927       |4.004       |3.293       |0.065       |
---------------------------------------------------
Valid: 	 mAP@50: 0.836296, mAP@75: 0.495590, mAP@90: 0.015652, Mean Loss: 13.289885
Noise level 1: 0
Noise level 3: 0.001


Eval: Valid: : 100%|██████████| 88/88 [00:06<00:00, 14.06it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
13.572      |6.080       |4.097       |3.325       |0.070       |
---------------------------------------------------
Valid: 	 mAP@50: 0.831938, mAP@75: 0.478995, mAP@90: 0.015734, Mean Loss: 13.571924
Noise level 1: 0
Noise level 3: 0.0025


...

---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
3737773.844 |1264591.633 |42236.012   |2078317.881 |352628.319  |
---------------------------------------------------
Valid: 	 mAP@50: 0.000000, mAP@75: 0.000000, mAP@90: 0.000000, Mean Loss: 3737773.843750
Noise level 1: 0.3
Noise level 3: 0.0025


Eval: Valid: : 100%|██████████| 88/88 [06:06<00:00,  4.17s/it]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
1318446.796 |462485.518  |13118.146   |741565.619  |101277.519  |
---------------------------------------------------
Valid: 	 mAP@50: 0.000000, mAP@75: 0.000000, mAP@90: 0.000000, Mean Loss: 1318446.796165
Noise level 1: 0.3
Noise level 3: 0.005


Eval: Valid: : 100%|██████████| 88/88 [07:38<00:00,  5.21s/it]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
1626451.210 |337474.608  |16246.152   |1250855.352 |21875.083   |
---------------------------------------------------
Valid: 	 mAP@50: 0.000000, mAP@75: 0.000000, mAP@90: 0.000000, Mean Loss: 1626451.210227


## **Test case 2 y case 3**

In [9]:
case1=False
case2=True
case3=True
noise_level_case_2  = [0, 0.05, 0.1, 0.15, 0.2, 0.3, 0.5] if case2 else [0]
noise_level_case_3 = [0, 0.001, 0.0025, 0.005] if case3 else [0]
test_noise(case1 = case1, noise_level_case_1 = [0], 
           case2 = case2, noise_level_case_2 = noise_level_case_2,
           case3 = case3, noise_level_case_3 = noise_level_case_3)

Test with case1: False, case2: True, case3: True
File case2_case3_results_noise.txt exists. Deleting...
=> Loading checkpoint
Model loaded: efficientnet/Adam_None_Airbus_256_BATCH_32_LR_0.0005/model/93_YOLO_best.pth.tar
Noise level 2: 0
Noise level 3: 0


Eval: Valid: : 100%|██████████| 88/88 [00:06<00:00, 14.47it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
13.087      |5.913       |4.003       |3.105       |0.065       |
---------------------------------------------------
Valid: 	 mAP@50: 0.836296, mAP@75: 0.495590, mAP@90: 0.015711, Mean Loss: 13.086825
Noise level 2: 0
Noise level 3: 0.001


Eval: Valid: : 100%|██████████| 88/88 [00:06<00:00, 12.89it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
13.509      |6.083       |4.218       |3.139       |0.069       |
---------------------------------------------------
Valid: 	 mAP@50: 0.828931, mAP@75: 0.467767, mAP@90: 0.016497, Mean Loss: 13.508835
Noise level 2: 0
Noise level 3: 0.0025


...

---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
2550.826    |528.342     |35.586      |1949.560    |37.337      |
---------------------------------------------------
Valid: 	 mAP@50: 0.000500, mAP@75: 0.000000, mAP@90: 0.000000, Mean Loss: 2550.825804
Noise level 2: 0.5
Noise level 3: 0.0025


Eval: Valid: : 100%|██████████| 88/88 [02:47<00:00,  1.90s/it]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
3774.642    |673.243     |48.962      |3010.240    |42.198      |
---------------------------------------------------
Valid: 	 mAP@50: 0.000296, mAP@75: 0.000000, mAP@90: 0.000000, Mean Loss: 3774.642477
Noise level 2: 0.5
Noise level 3: 0.005


Eval: Valid: : 100%|██████████| 88/88 [03:35<00:00,  2.45s/it]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
102838.904  |8676.896    |907.811     |92728.265   |525.932     |
---------------------------------------------------
Valid: 	 mAP@50: 0.000000, mAP@75: 0.000000, mAP@90: 0.000000, Mean Loss: 102838.904252


## **Test case 1, case 2 y case 3**

In [3]:
case1=True
case2=True
case3=True
noise_level_case_1  = [0, 0.01, 0.03, 0.05, 0.07, 0.08, 0.1, 0.125 ,0.15, 0.2, 0.3] if case1 else [0]
noise_level_case_2  = [0, 0.1, 0.15, 0.2] if case2 else [0]
noise_level_case_3 = [0, 0.001, 0.0025, 0.005] if case3 else [0]
test_noise(case1 = case1, noise_level_case_1 = noise_level_case_1, 
           case2 = case2, noise_level_case_2 = noise_level_case_2,
           case3 = case3, noise_level_case_3 = noise_level_case_3)

Test with case1: True, case2: True, case3: True
File case1_case2_case3_results_noise.txt exists. Deleting...
=> Loading checkpoint
Model loaded: efficientnet/Adam_None_Airbus_256_BATCH_32_LR_0.0005/model/93_YOLO_best.pth.tar
Noise level 1: 0
Noise level 2: 0
Noise level 3: 0


Eval: Valid: :   0%|          | 0/88 [00:00<?, ?it/s]Exception in thread Thread-6 (_pin_memory_loop):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.11/threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.11/site-packages/torch/utils/data/_utils/pin_memory.py", line 54, in _pin_memory_loop
    do_one_step()
  File "/opt/conda/lib/python3.11/site-packages/torch/utils/data/_utils/pin_memory.py", line 31, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/torch/multiprocessing/reductions.py", line 495, in rebuild_storage_fd
    fd = df.detach()
     

KeyboardInterrupt: 